In [ ]:
import pandas as pd
import json
import bs4
from bs4 import BeautifulSoup
import requests as r
import csv
import os

In [ ]:
#Scrape the FDIC page using beautifulsoup
#Since JavaScript isn't used to load the page, we can get away without having to use a browser (like Chromium or Firefox)
web_link ='https://www.fdic.gov/regulations/resources/rates/'
Get_DB_Data=r.get(web_link, verify=False)
convert_to_text=Get_DB_Data.text
soup=BeautifulSoup(convert_to_text, 'html.parser')

In [ ]:
table_data=[]
#Find the table on the webpage that contains the rate data and clean up some variables 
table = soup.find_all("table", { "class" : "fdic-table fdic-table-alt width-full" })
date_extract = soup.findAll("h2")[5]
date_extract_txt= str(date_extract)
date_extract2= date_extract_txt.replace("Monthly Rate Cap Information as of"," ")
date_extract3= date_extract2.replace("<h2>","")
date_extract4= date_extract3.replace("</h2>","")
date_extract5= date_extract4.replace(".","")
table1 = table[0]

In [ ]:
#download the results of the table row by row and throw it into a DataFrame
table_1_create=([[td.text for td in row.find_all("td")] for row in table1.select("tr")])

table_1_df=pd.DataFrame(table_1_create)
table_1_df.columns=['Deposit_Products', 'National_Deposit_Rate', 'National_Deposit_Rate_Cap_Adj','Treasury_Yield','Treasury_Yield_Rate_Cap_Adj','National_Rate_Cap']
table_1_df= table_1_df.drop(table_1_df.index[[0]])

In [ ]:
table_1_df['Date_of_Rate']= pd.to_datetime(date_extract5)

In [ ]:
#Throw the results into a file for output
import time
timestr = time.strftime("%Y%m%d")
filename = 'filename_' + timestr + '.csv'
table_1_df.to_csv(filename, index=False)